In [65]:
import importlib
import models
importlib.reload(models)

import pickle
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from statistics import median
import numpy as np
from numpy.linalg import slogdet

In [66]:
def max_degree(G):
    # Initialize max_degree
    max_degree = -1

    # Iterate over all nodes and their degrees
    for degree in G.degree():
        if degree[1] > max_degree:
            max_degree = degree[1]
    
    return max_degree

In [67]:
df = pd.read_csv("csv_files/model_one_desired_avg_deg.csv")
df = df[["num_vertices", "rand_seed"]].drop_duplicates()

new_vert = [200, 400, 600, 800, 1000, 1400, 1800] #, 1800, 2000, 2500]
for vert in new_vert:
    for seed in df["rand_seed"].unique()[0:2]:
        new_row = pd.DataFrame([{'num_vertices': vert, 'rand_seed': seed}])
        df = pd.concat([df, new_row], ignore_index=True)

In [68]:
# Check if planar and connected
planar = []
connected = []
avg_deg = []
median_deg = []
max_deg = []

#spanning tree information
num_nodes = []
tree_const = []
log_trees = []


for index, row in df.iterrows():
    n = int(row["num_vertices"])
    rs = int(row["rand_seed"])
    
    # CHOOSE MODEL
    model = "12b"
    G = models.model_eight_order_switched(n, rs, scaling_factor=0.5, remove_prob=0.14)

    # Planar?
    planar.append(nx.check_planarity(G)[0])
    
    # Connected?
    connected.append(nx.is_connected(G))

    # Calculate avg degree of graph
    avg_deg.append(2 * G.number_of_edges() / G.number_of_nodes())
    
    # Calculate median degree of graph
    degrees = sorted([degree for _, degree in G.degree()], reverse=False)
    median_deg.append(median(degrees))

    # Calculate max degree
    max_deg.append(max_degree(G))
    
    # calculate graph Laplacian
    # print(f'calculating model {model}:', n, rs)
    Lap = nx.laplacian_matrix(G).toarray()
    T = np.delete(Lap,1,0)
    T = np.delete(T,1,1)
    (sign, logabsdet) = slogdet(T)
    if (sign == 1):
        tree_const.append(logabsdet/n)
        num_nodes.append(n)
        log_trees.append(logabsdet)

In [69]:
model_df = df.assign(planar=planar, connected=connected, 
                            avg_deg=avg_deg, median_deg=median_deg, max_deg=max_deg)

In [70]:
# Calculate the average of planar, connected, median_deg, max_deg
columns_avg = model_df[['planar', 'connected', 'avg_deg', 'median_deg', 'max_deg']].mean()
columns_avg["avg_st_con"] = st_cons=np.mean(tree_const)
columns_avg.round(2)

planar         0.00
connected      1.00
avg_deg        5.36
median_deg     5.35
max_deg       10.14
avg_st_con     1.41
dtype: float64

In [71]:
# avg deg 5.4
# st con 1.33

In [72]:
# Change to True to save the lists as pickles
if True:
    with open(f'pkls/num_nodes_m{model}.pkl', 'wb') as f:
        pickle.dump(num_nodes, f)
    with open(f'pkls/tree_const_m{model}.pkl', 'wb') as f:
        pickle.dump(tree_const, f)
    with open(f'pkls/log_trees_m{model}.pkl', 'wb') as f:
        pickle.dump(log_trees, f)